In [1]:
import logging
import pathlib
import pprint
import sys

import joblib
import numpy as np
import pandas as pd
import toml
import torch

sys.path.append("../ML_utils/")

from parameter_set import parameter_set
from parameters import Parameters
from train_optimized_model import train_optimized_model

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False
if in_notebook:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

In [2]:
# read in the data
sc_file_path = pathlib.Path("../results/cleaned_sc_profile.parquet").resolve(
    strict=True
)
sc_endpoint_file_path = pathlib.Path(
    "../results/cleaned_endpoint_sc_profile.parquet"
).resolve(strict=True)

data_split_file_path = pathlib.Path("../results/data_splits.parquet").resolve(
    strict=True
)

sc_profile = pd.read_parquet(sc_file_path)
sc_endpoint_profile = pd.read_parquet(sc_endpoint_file_path)
data_split_df = pd.read_parquet(data_split_file_path)
print(f"sc_profile shape: {sc_profile.shape}")
print(f"sc_endpoint_profile shape: {sc_endpoint_profile.shape}")
print(f"data_split_df shape: {data_split_df.shape}")
data_split_df.head()

sc_profile shape: (182804, 2376)
sc_endpoint_profile shape: (11340, 368)
data_split_df shape: (14926, 3)


,index,data_split,data_x_or_y
0,7440,train_gt,X
1,7461,train_gt,X
2,7463,train_gt,X
3,7468,train_gt,X
4,7479,train_gt,X


In [3]:
# keep only the last timepoint
sc_profile["Metadata_Time"] = sc_profile["Metadata_Time"].astype("float64")
sc_profile = sc_profile[
    sc_profile["Metadata_Time"] == sc_profile["Metadata_Time"].max()
]
# drop Na values
sc_profile.dropna(inplace=True)
print(f"sc_profile shape after dropping NaN: {sc_profile.shape}")
sc_endpoint_profile.dropna(inplace=True)
print(f"sc_endpoint_profile shape after dropping NaN: {sc_endpoint_profile.shape}")
# hardcode the features that should exist in the y data
# this will be replaced in the future by an arg or config passed through
selected_y_features = ["Cells_Intensity_MeanIntensityEdge_AnnexinV"]
metadata_y_features = [x for x in sc_endpoint_profile.columns if "Metadata_" in x]
sc_endpoint_profile = sc_endpoint_profile[metadata_y_features + selected_y_features]
print(
    f"sc_endpoint_profile shape after selecting features: {sc_endpoint_profile.shape}"
)

sc_profile shape after dropping NaN: (14237, 2376)
sc_endpoint_profile shape after dropping NaN: (11136, 368)
sc_endpoint_profile shape after selecting features: (11136, 26)


In [4]:
profile_data_splits_df = data_split_df.loc[data_split_df["data_x_or_y"] == "X"]
profile_data_splits_df
endpoint_data_splits_df = data_split_df.loc[data_split_df["data_x_or_y"] == "y"]
endpoint_data_splits_df
# replace the index with the index column
profile_data_splits_df = profile_data_splits_df.set_index(
    "index", drop=True, verify_integrity=True
)
endpoint_data_splits_df = endpoint_data_splits_df.set_index(
    "index", drop=True, verify_integrity=True
)
# remove the index name from profile_data_splits_df
profile_data_splits_df.index.name = None
# remove the index name from endpoint_data_splits_df
endpoint_data_splits_df.index.name = None

In [5]:
# get only indexes from sc_profile that are in the train_gt split
train_gt_X = sc_profile.loc[
    sc_profile.index.isin(
        profile_data_splits_df.loc[
            profile_data_splits_df["data_split"] == "train_gt"
        ].index
    )
]
val_gt_X = sc_profile.loc[
    sc_profile.index.isin(
        profile_data_splits_df.loc[
            profile_data_splits_df["data_split"] == "val_gt"
        ].index
    )
]
train_gt_y = sc_endpoint_profile.loc[
    sc_endpoint_profile.index.isin(
        endpoint_data_splits_df.loc[
            endpoint_data_splits_df["data_split"] == "train_gt"
        ].index
    )
]
val_gt_y = sc_endpoint_profile.loc[
    sc_endpoint_profile.index.isin(
        endpoint_data_splits_df.loc[
            endpoint_data_splits_df["data_split"] == "val_gt"
        ].index
    )
]

# assertion checks
assert train_gt_X.shape[0] == train_gt_y.shape[0]
assert val_gt_X.shape[0] == val_gt_y.shape[0]
assert train_gt_X.shape[1] == val_gt_X.shape[1]
assert train_gt_y.shape[1] == val_gt_y.shape[1]

In [6]:
# get metadata
metadata_X_cols = [x for x in train_gt_X.columns if "Metadata_" in x]
metadata_y_cols = [x for x in train_gt_y.columns if "Metadata_" in x]


train_gt_X_metadata = train_gt_X[metadata_X_cols]
train_gt_X.drop(columns=metadata_X_cols, inplace=True)
train_gt_y_metadata = train_gt_y[metadata_y_cols]
train_gt_y.drop(columns=metadata_y_cols, inplace=True)
val_gt_X_metadata = val_gt_X[metadata_X_cols]
val_gt_X.drop(columns=metadata_X_cols, inplace=True)
val_gt_y_metadata = val_gt_y[metadata_y_cols]
val_gt_y.drop(columns=metadata_y_cols, inplace=True)

/tmp/ipykernel_1345311/735914267.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_gt_X.drop(columns=metadata_X_cols, inplace=True)
/tmp/ipykernel_1345311/735914267.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_gt_y.drop(columns=metadata_y_cols, inplace=True)
/tmp/ipykernel_1345311/735914267.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_gt_X.drop(columns=metadata_X_cols, inplace=True)
/tmp/ipykernel_13453

In [7]:
# shuffle the data
shuffled_train_gt_X = train_gt_X.copy()
for col in shuffled_train_gt_X.columns:
    if col.startswith("Metadata_"):
        continue
    shuffled_train_gt_X[col] = np.random.permutation(shuffled_train_gt_X[col].values)
shuffled_val_gt_X = val_gt_X.copy()
for col in shuffled_val_gt_X.columns:
    if col.startswith("Metadata_"):
        continue
    shuffled_val_gt_X[col] = np.random.permutation(shuffled_val_gt_X[col].values)

In [8]:
# number of input features
n_features = train_gt_X.shape[1]
# number of output features
n_outputs = train_gt_y.shape[1]
# number of metadata features
n_metadata_features = train_gt_X_metadata.shape[1]

print(f"n_features: {n_features}")
print(f"n_outputs: {n_outputs}")
print(f"n_metadata_features: {n_metadata_features}")

n_features: 2338
n_outputs: 1
n_metadata_features: 38


In [9]:
params = Parameters()
ml_configs = toml.load("../ML_utils/regression_class_config.toml")
mlp_params = parameter_set(params, ml_configs)
mlp_params.IN_FEATURES = n_features
mlp_params.OUT_FEATURES = n_outputs

In [10]:
print(train_gt_X.shape, train_gt_y.shape)
print(val_gt_X.shape, val_gt_y.shape)

(367, 2338) (367, 1)
(49, 2338) (49, 1)


In [11]:
X_train = torch.tensor(train_gt_X.values, dtype=torch.float32)
y_train = torch.tensor(train_gt_y.values, dtype=torch.float32)
X_val = torch.tensor(val_gt_X.values, dtype=torch.float32)
y_val = torch.tensor(val_gt_y.values, dtype=torch.float32)

In [12]:
# get the dtypes of the data
print(f"X_train dtypes: {X_train.dtype}")
print(f"y_train dtypes: {y_train.dtype}")
print(f"X_val dtypes: {X_val.dtype}")
print(f"y_val dtypes: {y_val.dtype}")

X_train dtypes: torch.float32
y_train dtypes: torch.float32
X_val dtypes: torch.float32
y_val dtypes: torch.float32


In [13]:
# produce data objects for train, val and test datasets
train_data = torch.utils.data.TensorDataset(X_train, y_train)
val_data = torch.utils.data.TensorDataset(X_val, y_val)


# convert data class into a dataloader to be compatible with pytorch
train_loader = torch.utils.data.DataLoader(
    dataset=train_data, batch_size=mlp_params.HYPERPARAMETER_BATCH_SIZE, shuffle=True
)
valid_loader = torch.utils.data.DataLoader(
    dataset=val_data, batch_size=mlp_params.HYPERPARAMETER_BATCH_SIZE, shuffle=False
)

In [14]:
# call the optimized training model
(
    train_loss,
    valid_loss,
    epochs_ran,
    model,
) = train_optimized_model(
    EPOCHS=mlp_params.TRAIN_EPOCHS,
    train_loader=train_loader,
    valid_loader=valid_loader,
    params=mlp_params,
    model_name="Cells_Intensity_MeanIntensityEdge_AnnexinV",
    shuffle=False,
)

SGD
Epoch 0: Validation loss decreased (inf --> 0.554434).  Saving model ...
	 Train_Loss: 1.1243 Val_Loss: 0.5544  BEST VAL Loss: 0.5544 

Epoch 1: Validation loss decreased (0.554434 --> 0.554351).  Saving model ...
	 Train_Loss: 1.1234 Val_Loss: 0.5544  BEST VAL Loss: 0.5544 

Epoch 2: Validation loss decreased (0.554351 --> 0.554269).  Saving model ...
	 Train_Loss: 1.1230 Val_Loss: 0.5543  BEST VAL Loss: 0.5543 

Epoch 3: Validation loss decreased (0.554269 --> 0.554187).  Saving model ...
	 Train_Loss: 1.1228 Val_Loss: 0.5542  BEST VAL Loss: 0.5542 

Epoch 4: Validation loss decreased (0.554187 --> 0.554107).  Saving model ...
	 Train_Loss: 1.1227 Val_Loss: 0.5541  BEST VAL Loss: 0.5541 

Epoch 5: Validation loss decreased (0.554107 --> 0.554028).  Saving model ...
	 Train_Loss: 1.1227 Val_Loss: 0.5540  BEST VAL Loss: 0.5540 

Epoch 6: Validation loss decreased (0.554028 --> 0.553950).  Saving model ...
	 Train_Loss: 1.1225 Val_Loss: 0.5540  BEST VAL Loss: 0.5540 

Epoch 7: Valid